In [2]:
import zipfile
with zipfile.ZipFile('important_words.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Loading Dataset

In [3]:
import pandas as pd
import numpy as np

In [4]:
products=pd.read_csv('amazon_baby_subset.csv')
products.head()

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1


In [12]:
products.iloc[0:10,:]
products = products.fillna({'review':''})

# Removing Punctuations

In [13]:
import string
table = str.maketrans(dict.fromkeys(string.punctuation))
def remove_punc(s):
    return str(s).translate(table)
products['review_clean'] = products['review'].apply(remove_punc)
products.head()

,name,review,rating,sentiment,review_clean
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...


# Loadig json file containing important words

In [14]:
import json
with open('important_words.json') as imp:
    important_words = json.load(imp)

In [15]:
important_words

['baby',
 'one',
 'great',
 'love',
 'use',
 'would',
 'like',
 'easy',
 'little',
 'seat',
 'old',
 'well',
 'get',
 'also',
 'really',
 'son',
 'time',
 'bought',
 'product',
 'good',
 'daughter',
 'much',
 'loves',
 'stroller',
 'put',
 'months',
 'car',
 'still',
 'back',
 'used',
 'recommend',
 'first',
 'even',
 'perfect',
 'nice',
 'bag',
 'two',
 'using',
 'got',
 'fit',
 'around',
 'diaper',
 'enough',
 'month',
 'price',
 'go',
 'could',
 'soft',
 'since',
 'buy',
 'room',
 'works',
 'made',
 'child',
 'keep',
 'size',
 'small',
 'need',
 'year',
 'big',
 'make',
 'take',
 'easily',
 'think',
 'crib',
 'clean',
 'way',
 'quality',
 'thing',
 'better',
 'without',
 'set',
 'new',
 'every',
 'cute',
 'best',
 'bottles',
 'work',
 'purchased',
 'right',
 'lot',
 'side',
 'happy',
 'comfortable',
 'toy',
 'able',
 'kids',
 'bit',
 'night',
 'long',
 'fits',
 'see',
 'us',
 'another',
 'play',
 'day',
 'money',
 'monitor',
 'tried',
 'thought',
 'never',
 'item',
 'hard',
 'plasti

# Word_count of each word

In [27]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))


**Quiz Question**. How many reviews contain the word perfect?

In [39]:
products[products['perfect']>0].shape[0]


2955

# Forming a feature matrix

In [43]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = np.array(features_frame)
    label_sarray = dataframe[label]
    label_array = np.array(label_sarray)
    return(feature_matrix, label_array)

In [44]:
feature_matrix=get_numpy_data(products,important_words,'sentiment')[0]
label_array=get_numpy_data(products,important_words,'sentiment')[1]

In [48]:
feature_matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [50]:
label_array

array([ 1,  1,  1, ..., -1, -1, -1])

**Quiz Question**: How many features are there in the feature_matrix?
*194*


**Quiz Question**: Assuming that the intercept is present, how does the number of features in feature_matrix relate to the number of features in the logistic regression model?


*-1*

In [78]:
import math
def predict_probablity(feature_matrix,coeff):
    p=np.dot(feature_matrix,coeff)
    p=1/(1+np.exp(-p))
    return p

In [79]:
def feature_derivative(errors, feature):     
    derivative =np.dot(errors,feature)
    return derivative

In [80]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

In [83]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(0,max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probablity(feature_matrix,coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        ft=np.transpose(feature_matrix)
        derivatives=np.dot(ft,errors)
        coefficients=np.array(coefficients)+step_size*np.array(derivatives)
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print ('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [84]:
initial_wt=np.array([0]*194)
initial_wt.reshape(194,1)
coeff=logistic_regression(feature_matrix,label_array,initial_wt,1e-7,301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

**Quiz question**: As each iteration of gradient ascent passes, does the log likelihood increase or decrease?

In [85]:
print (coeff)

[ 5.16220157e-03  1.55656966e-02 -8.50204675e-03  6.65460842e-02
  6.58907629e-02  5.01743882e-03 -5.38601484e-02 -3.50488413e-03
  6.47945868e-02  4.54356263e-02  3.98353364e-03  2.00775410e-02
  3.01350011e-02 -2.87115530e-02  1.52161964e-02  2.72592062e-04
  1.19448177e-02 -1.82461935e-02 -1.21706420e-02 -4.15110334e-02
  2.76820391e-03  1.77031999e-02 -4.39700067e-03  4.49764014e-02
  9.90916464e-03  8.99239081e-04 -1.36219516e-03  1.26859357e-02
  8.26466695e-03 -2.77426972e-02  6.10128809e-04  1.54084501e-02
 -1.32134753e-02 -3.00512492e-02  2.97399371e-02  1.84087080e-02
  2.86178752e-03 -1.05768015e-02 -6.57350362e-04 -1.01476555e-02
 -4.79579528e-03  7.50891810e-03  4.27938289e-03  3.06785501e-03
 -2.20317661e-03  9.57273354e-03  9.91666827e-05 -1.98462567e-02
  1.75702722e-02  1.55478612e-03 -1.77375440e-02  9.78324102e-03
  1.17031606e-02 -7.35345937e-03 -6.08714030e-03  6.43766808e-03
  1.07159665e-02 -3.05345476e-03  7.17190727e-03  5.73320003e-03
  4.60661876e-03 -5.20588

In [105]:
scores=np.dot(feature_matrix,coeff)
scores.reshape(scores.shape[0],1)
c=0
for i in scores:
    if i>0:
        c=c+1
print (c)        

53072
25126


**Quiz question**: How many reviews were predicted to have positive sentiment?

# Measuring Accuracy

In [119]:
correct=0
senti=np.array(products['sentiment'])
l=0
for i in range(0,len(scores)):
#     print (products[products.iloc[[i],[3]]])
    if scores[i]>0 and senti[i]==1:
        correct=correct+1
    if scores[i]<=0 and senti[i]==-1:
        l=l+1;
        correct=correct+1
accuracy=correct/len(scores)
print (accuracy)

0.7518653904130238


In [124]:
p=[]
for i in range(1,194):
    p.append([coeff[i],important_words[i-1]])
p.sort()    

**Quiz question**: Which word is not present in the top 10 "most negative" words?

In [126]:
for i in range(0,10):
    print(p[i])

[-0.05386014844520313, 'would']
[-0.04151103339210889, 'product']
[-0.038982037286487116, 'money']
[-0.03306951529475273, 'work']
[-0.030051249236035808, 'even']
[-0.028978976142317068, 'disappointed']
[-0.028711552980192585, 'get']
[-0.027742697230661327, 'back']
[-0.026592778462247283, 'return']
[-0.024482100545891717, 'monitor']


**Quiz question**: Which word is not present in the top 10 "most positive" words?

In [127]:
p.sort(reverse=True)

In [128]:
for i in range(0,10):
    print(p[i])

[0.06654608417045771, 'great']
[0.06589076292212324, 'love']
[0.0647945868025784, 'easy']
[0.04543562630842137, 'little']
[0.04497640139490604, 'loves']
[0.030135001092107077, 'well']
[0.02973993710496846, 'perfect']
[0.020077541034775385, 'old']
[0.01840870799526899, 'nice']
[0.017703199905701697, 'daughter']
